In [91]:
import pandas as pd
import re


In [127]:
def parse_file(namefile):
    
    df_tmp = pd.DataFrame(columns = ['type','fold','RMSE','MAE','R2','features'])

    with open(namefile, "r") as f:
        file_contents = f.read()

    rmse = re.findall(r"RMSE test (-?[\d.]+)", file_contents)
    mae = re.findall(r"MAE test (-?[\d.]+)", file_contents)
    r2 = re.findall(r"R2 test (-?[\d.]+)", file_contents)
    
    pattern = r"_(.*?)\.log"
    match = re.search(pattern, namefile)
    if match:
        type_ = match.group(1)

    df_tmp['RMSE'] = pd.to_numeric(pd.Series(rmse))
    df_tmp['MAE'] = pd.to_numeric(pd.Series(mae))
    df_tmp['R2'] = pd.to_numeric(pd.Series(r2))
    df_tmp['type'] = pd.Series([type_]*len(mae))
    df_tmp['fold'] = pd.to_numeric(pd.Series([1,2,3,4,5]*(4)))
    
    series_values = ['all'] * 5 + ['classic'] * 5 + ['pupil'] * 5 + ['ou'] * 5
    series_index = ['A', 'B', 'C', 'D', 'E'] * 4

    df_tmp['features'] = pd.Series(series_values)

    return df_tmp

In [128]:
df_mlp = parse_file("output_mlp.log")

In [129]:
df_gp = parse_file("output_gp.log")

In [130]:
df_rf = parse_file("output_rf.log")

In [131]:
df_svm = parse_file("output_svm.log")

In [132]:
df = pd.concat((df_mlp,df_gp,df_rf,df_svm))

In [133]:
df.to_csv('metrics_results.csv', index=False)

In [134]:
df

,type,fold,RMSE,MAE,R2,features
0,mlp,1,6.645393,4.734410,0.299397,all
1,mlp,2,6.241332,4.531311,0.382342,all
2,mlp,3,6.652367,4.861051,0.298304,all
3,mlp,4,6.237107,4.401830,0.382868,all
4,mlp,5,7.016061,4.540525,0.220856,all
...,...,...,...,...,...,...
15,svm,1,7.958160,5.568466,-0.004745,ou
16,svm,2,7.877014,5.514766,0.016178,ou
17,svm,3,7.917876,5.545049,0.005937,ou
18,svm,4,7.953239,5.544676,-0.003459,ou


In [135]:
df_mlp = df[df['type']=='mlp']

In [150]:
df_mlp[['RMSE','MAE','R2','features']].groupby(['features']).mean().sort_values('RMSE')

,RMSE,MAE,R2
features,,,
all,6.558452,4.613825,0.316754
pupil,7.232298,5.237873,0.170743
classic,7.882491,6.147866,0.014943
ou,20.569459,6.001027,-16.797265


In [140]:
df_gp = df[df['type']=='gp']

In [149]:
df_gp[['RMSE','MAE','R2','features']].groupby(['features']).mean().sort_values('RMSE')

,RMSE,MAE,R2
features,,,
all,7.184484,5.163235,0.181664
pupil,7.242580,5.226220,0.168384
ou,7.787820,5.972065,0.038463
classic,7.892509,6.085696,0.012439


In [142]:
df_rf = df[df['type']=='rf']

In [148]:
df_rf[['RMSE','MAE','R2','features']].groupby(['features']).mean().sort_values('RMSE')

,RMSE,MAE,R2
features,,,
all,6.386239,4.571497,0.353359
pupil,7.209608,5.214524,0.175942
ou,7.233362,5.603272,0.170476
classic,7.790790,6.029662,0.037729


In [144]:
df_svm = df[df['type']=='svm']

In [147]:
df_svm[['RMSE','MAE','R2','features']].groupby(['features']).mean().sort_values('RMSE')

,RMSE,MAE,R2
features,,,
all,7.600923,5.050673,0.084058
ou,7.925744,5.542345,0.004091
pupil,8.137482,5.414734,-0.049829
classic,8.159746,5.792489,-0.055568
